In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from datetime import timedelta
import requests
from pandas.tseries.offsets import BDay

In [2]:
def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def get_top_outliers(df, n=10):
    return df.nlargest(n, 'abs_daily_return')

# Define the API key and base URL
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

# Define the currency pairs and years
pair = "C:USDCHF"
years = range(2010, 2024)

# Initialize DataFrames
stats_data = pd.DataFrame(columns=['Year', 'Mean', 'Variance', 'Skewness', 'Kurtosis'])
full_data = pd.DataFrame()
outliers_data = pd.DataFrame()

# Loop over each year
for year in years:
	# Format the API endpoint
	start_date = f'{year}-01-01'
	end_date = f'{year}-12-31'
	url = f"https://api.polygon.io/v2/aggs/ticker/{pair}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"

	# Make the API request
	response = requests.get(url)
	data = response.json()

	# Check if the request was successful
	if response.status_code == 200 and 'results' in data:
		# Load data into a DataFrame
		df = pd.DataFrame(data['results'])
        # Converting timestamps
		df['date'] = pd.to_datetime(df['t'], unit='ms')
		df.drop(columns=['t'], inplace=True)
        # Calculating returns
		df = calculate_daily_returns(df)

		# Calculate basic statistical properties of the daily returns
		returns = df['daily_return']
		mean_return = returns.mean()
		variance_return = returns.var()
		skewness_return = skew(returns)
		kurtosis_return = kurtosis(returns)

		# Collect stats data by year
		year_stats = pd.DataFrame([[year, mean_return, variance_return, skewness_return, kurtosis_return]],
									columns=['Year', 'Mean', 'Variance', 'Skewness', 'Kurtosis'])
		stats_data = pd.concat([stats_data, year_stats], ignore_index=True)

		# Append the data to the full_data DataFrame for the current currency pair
		df['year'] = year
		full_data = pd.concat([full_data, df], ignore_index=True)

		# Find the top 10 outliers based on absolute values of the daily return value
		df['abs_daily_return'] = df['c'].abs()
		outliers = df.nlargest(10, 'abs_daily_return')

		# Append outliers to the outliers_data DataFrame for the current currency pair
		outliers['year'] = year
		outliers_data = pd.concat([outliers_data, outliers], ignore_index=True)

# Sort the data by date so that visualization could be done in a better way
sorted_full_data = full_data.sort_values(by="date")
sorted_outliers_data = outliers_data.sort_values(by="date")

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/4141882298.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_data = pd.concat([stats_data, year_stats], ignore_index=True)


In [3]:
sorted_full_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year
0,2359,1.0351,1.036260,1.036040,1.036880,1.033780,2359,2010-01-03,NaN,NaN,NaN,2010
1,69913,1.0329,1.036060,1.028910,1.041960,1.026210,69913,2010-01-04,1.03604,-0.006882,0.006882,2010
2,76822,1.0304,1.029000,1.033410,1.035960,1.025150,76822,2010-01-05,1.02891,0.004374,0.004374,2010
3,81057,1.0321,1.033410,1.027460,1.037830,1.024800,81057,2010-01-06,1.03341,-0.005758,0.005758,2010
4,72488,1.0321,1.027440,1.033700,1.037050,1.024140,72488,2010-01-07,1.02746,0.006073,0.006073,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
4399,112192,0.8555,0.856000,0.853720,0.857960,0.852800,112192,2023-12-26,0.85581,-0.002442,0.002442,2023
4400,174820,0.8495,0.853590,0.842500,0.854870,0.840600,174820,2023-12-27,0.85372,-0.013142,0.013142,2023
4401,196178,0.8392,0.842400,0.843370,0.845145,0.833000,196178,2023-12-28,0.84250,0.001033,0.001033,2023
4402,214081,0.8403,0.843460,0.840710,0.843560,0.835400,214081,2023-12-29,0.84337,-0.003154,0.003154,2023


In [4]:
sorted_outliers_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year
2,116306,1.1565,1.15104,1.16024,1.16191,1.14913,116306,2010-05-24,1.151030,0.008002,1.16024,2010
6,176660,1.1625,1.16023,1.15739,1.16950,1.15178,176660,2010-05-25,1.160240,-0.002456,1.15739,2010
4,135028,1.1579,1.15739,1.15873,1.16230,1.15139,135028,2010-05-26,1.157390,0.001158,1.15873,2010
3,103597,1.1541,1.15223,1.15927,1.15929,1.14801,103597,2010-05-28,1.152140,0.006188,1.15927,2010
5,4795,1.1580,1.15828,1.15740,1.15920,1.15709,4795,2010-05-30,1.159270,-0.001613,1.15740,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
137,168297,0.9389,0.94201,0.93949,0.94290,0.93480,168297,2023-03-01,0.941868,-0.002524,0.93949,2023
132,164385,0.9419,0.93950,0.94180,0.94400,0.93890,164385,2023-03-02,0.939490,0.002459,0.94180,2023
138,2794,0.9366,0.93590,0.93680,0.93698,0.93550,2794,2023-03-05,0.935854,0.001011,0.93680,2023
130,172706,0.9356,0.93069,0.94223,0.94260,0.92830,172706,2023-03-07,0.930550,0.012552,0.94223,2023


In [5]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data:", response.status_code, response.text)
        return None
    data = response.json()
    if 'results' not in data:
        print("No 'results' key in response:", data)
        return None
    return data

def calculate_daily_returns_threshold(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    # print(df['daily_return'])
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def convert_timestamps(df):
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    return df


def fetch_daily_return_prior(symbol, current_date, previous_date, api_key):
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{previous_date}/{current_date}?adjusted=true&sort=asc&apiKey={api_key}'
    data = fetch_data(url)
    if data and 'results' in data and len(data['results']) > 0:
        df = pd.DataFrame(data['results'])
        # print(df)
        df = convert_timestamps(df)
        # print(f"converted: {df}")
        df = df.loc[::-1].reset_index(drop=True)
        df = calculate_daily_returns_threshold(df)
        print(f"daily return: {df}")
        # print(f"daily_return {df['abs_daily_return']}")
        if len(df['abs_daily_return']) == 2:
            return df['abs_daily_return'].iloc[1]
    return None


def find_prior_outlier_threshold(df, symbol, api_key):
    df = df.sort_values(by='date').reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'])

    results = []

    for index, row in df.iterrows():
        current_date = row['date']
        current_return = row['abs_daily_return']
        print(f"outlier date: {current_date}")
        while True:
            # print(f"current_return: {current_return}")
            previous_date = current_date - pd.DateOffset(days=1)
            # print(f"previous date: {previous_date}")
            current_date_str = current_date.strftime('%Y-%m-%d')
            previous_date_str = previous_date.strftime('%Y-%m-%d')
            previous_return = fetch_daily_return_prior(symbol, current_date_str, previous_date_str, api_key)
            # print(f"previous return: {previous_return}")
            if previous_return is not None:
                if (current_return - previous_return) < 0:

                    days_difference = (row['date'] - previous_date).days
                    # print(current_return - previous_return)
                    results.append({
                        'outlier_date': row['date'],
                        'prior_outlier_threshold_date': previous_date,
                        'days_difference': days_difference
                    })
                    # print()
                    break
                else:
                    current_date = previous_date
                    current_return = previous_return
            else:
                # print(f"Failed to fetch data for {previous_date_str}")
                # break
                current_date = previous_date

    return pd.DataFrame(results)


# Applying the corrected function to the data
prior_outlier_thresholds = find_prior_outlier_threshold(sorted_outliers_data, pair, api_key)

outlier date: 2010-05-24 00:00:00
daily return:         v      vw        o        c        h        l       n       date  \
0  116306  1.1565  1.15104  1.16024  1.16191  1.14913  116306 2010-05-24   
1    5423  1.1501  1.14963  1.15103  1.15160  1.14885    5423 2010-05-23   

   prev_close  daily_return  abs_daily_return  
0         NaN           NaN               NaN  
1     1.16024     -0.007938          0.007938  
daily return:       v      vw        o        c       h        l     n       date  \
0  5423  1.1501  1.14963  1.15103  1.1516  1.14885  5423 2010-05-23   

   prev_close  daily_return  abs_daily_return  
0         NaN           NaN               NaN  
daily return:         v      vw        o        c        h        l       n       date  \
0  148195  1.1499  1.14874  1.14911  1.15797  1.14496  148195 2010-05-21   

   prev_close  daily_return  abs_daily_return  
0         NaN           NaN               NaN  
daily return:         v      vw        o        c        h     

In [6]:
def fetch_daily_return_post(symbol, current_date, post_date, api_key):
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{current_date}/{post_date}?adjusted=true&sort=asc&apiKey={api_key}'
    data = fetch_data(url)
    if data and 'results' in data and len(data['results']) > 0:
        df = pd.DataFrame(data['results'])
        # print(df)
        df = convert_timestamps(df)
        # print(f"converted: {df}")
        df = calculate_daily_returns_threshold(df)
        # print(f"daily return: {df}")
        # print(f"daily_return {df['abs_daily_return']}")
        if len(df['abs_daily_return']) == 2:
            return df['abs_daily_return'].iloc[1]
    return None


def find_post_outlier_threshold(df, symbol, api_key):
    df = df.sort_values(by='date').reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'])

    results = []

    for index, row in df.iterrows():
        current_date = row['date']
        current_return = row['abs_daily_return']
        # print(f"outlier date: {current_date}")
        while True:
            # print(f"current_return: {current_return}")
            post_date = current_date + pd.DateOffset(days=1)
            # print(f"previous date: {previous_date}")
            current_date_str = current_date.strftime('%Y-%m-%d')
            post_date_str = post_date.strftime('%Y-%m-%d')
            post_return = fetch_daily_return_post(symbol, current_date_str, post_date_str, api_key)
            # print(f"post return: {post_return}")
            if post_return is not None:
                if (current_return - post_return) < 0:

                    days_difference = (post_date - row['date']).days
                    # print(current_return - post_return)
                    results.append({
                        'outlier_date': row['date'],
                        'post_outlier_threshold_date': post_date,
                        'days_difference': days_difference
                    })
                    # print()
                    break
                else:
                    current_date = post_date
                    current_return = post_return
            else:
                # print(f"Failed to fetch data for {post_date_str}")
                # break
                current_date = post_date

    return pd.DataFrame(results)

post_outlier_thresholds = find_post_outlier_threshold(sorted_outliers_data, pair, api_key)

No 'results' key in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '763ccd2a3de129890e66c7e8c513e216'}


In [7]:
# Finding the largest days_difference
max_days_difference = prior_outlier_thresholds['days_difference'].max()
print(f"Largest days_difference: {max_days_difference}")
prior_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
prior_outlier_thresholds

Largest days_difference: 8


,date,prior_outlier_threshold_date,days_difference
0,2010-05-24,2010-05-19,5
1,2010-05-25,2010-05-23,2
2,2010-05-26,2010-05-24,2
3,2010-05-28,2010-05-24,4
4,2010-05-30,2010-05-24,6
...,...,...,...
135,2023-03-01,2023-02-27,2
136,2023-03-02,2023-02-28,2
137,2023-03-05,2023-03-02,3
138,2023-03-07,2023-03-02,5


In [8]:
post_outlier_thresholds

,outlier_date,post_outlier_threshold_date,days_difference
0,2010-05-24,2010-05-27,3
1,2010-05-25,2010-05-27,2
2,2010-05-26,2010-05-28,2
3,2010-05-28,2010-06-03,6
4,2010-05-30,2010-06-03,4
...,...,...,...
135,2023-03-01,2023-03-03,2
136,2023-03-02,2023-03-05,3
137,2023-03-05,2023-03-07,2
138,2023-03-07,2023-03-09,2


In [9]:
prior_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
sorted_outliers_data = pd.merge(
    sorted_outliers_data,
    prior_outlier_thresholds[['date', 'days_difference']],
    on = 'date',
    how='left'
)
sorted_outliers_data.rename(columns={'days_difference':'pre_threshold'}, inplace=True)

post_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
sorted_outliers_data = pd.merge(
    sorted_outliers_data,
    post_outlier_thresholds[['date', 'days_difference']],
    on = 'date',
)
sorted_outliers_data.rename(columns={'days_difference':'post_threshold'}, inplace=True)

In [10]:
sorted_outliers_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year,pre_threshold,post_threshold
0,116306,1.1565,1.15104,1.16024,1.16191,1.14913,116306,2010-05-24,1.151030,0.008002,1.16024,2010,5,3
1,176660,1.1625,1.16023,1.15739,1.16950,1.15178,176660,2010-05-25,1.160240,-0.002456,1.15739,2010,2,2
2,135028,1.1579,1.15739,1.15873,1.16230,1.15139,135028,2010-05-26,1.157390,0.001158,1.15873,2010,2,2
3,103597,1.1541,1.15223,1.15927,1.15929,1.14801,103597,2010-05-28,1.152140,0.006188,1.15927,2010,4,6
4,4795,1.1580,1.15828,1.15740,1.15920,1.15709,4795,2010-05-30,1.159270,-0.001613,1.15740,2010,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,168297,0.9389,0.94201,0.93949,0.94290,0.93480,168297,2023-03-01,0.941868,-0.002524,0.93949,2023,2,2
136,164385,0.9419,0.93950,0.94180,0.94400,0.93890,164385,2023-03-02,0.939490,0.002459,0.94180,2023,2,3
137,2794,0.9366,0.93590,0.93680,0.93698,0.93550,2794,2023-03-05,0.935854,0.001011,0.93680,2023,3,2
138,172706,0.9356,0.93069,0.94223,0.94260,0.92830,172706,2023-03-07,0.930550,0.012552,0.94223,2023,5,2


In [11]:
# Convert dates in dataset to datetime objects
sorted_outliers_data['date'] = pd.to_datetime(sorted_outliers_data['date'])
mean_daily_return = sorted_outliers_data['daily_return'].mean()

# date_ranges = pd.DataFrame({
#     "start_date": sorted_outliers_data['date'] - BDay(3),
#     "end_date": sorted_outliers_data['date'] + BDay(3),
#     "outlier_date": sorted_outliers_data['date'],
#     "year": sorted_outliers_data['year']
# })

date_ranges = pd.DataFrame({
    "start_date": sorted_outliers_data.apply(lambda row: row['date'] - BDay(row['pre_threshold']), axis=1),
    "end_date": sorted_outliers_data.apply(lambda row: row['date'] + BDay(row['post_threshold']), axis=1),
    "outlier_date": sorted_outliers_data['date'],
    "year": sorted_outliers_data['year']
})

date_ranges

,start_date,end_date,outlier_date,year
0,2010-05-17,2010-05-27,2010-05-24,2010
1,2010-05-21,2010-05-27,2010-05-25,2010
2,2010-05-24,2010-05-28,2010-05-26,2010
3,2010-05-24,2010-06-07,2010-05-28,2010
4,2010-05-21,2010-06-03,2010-05-30,2010
...,...,...,...,...
135,2023-02-27,2023-03-03,2023-03-01,2023
136,2023-02-28,2023-03-07,2023-03-02,2023
137,2023-03-01,2023-03-07,2023-03-05,2023
138,2023-02-28,2023-03-09,2023-03-07,2023


In [12]:
def classify_recovery_range(daily_data, outlier_date, mean_daily_return):
    pre_threshold = sorted_outliers_data.loc[sorted_outliers_data['date'] == outlier_date, 'pre_threshold'].values[0]
    post_threshold = sorted_outliers_data.loc[sorted_outliers_data['date'] == outlier_date, 'post_threshold'].values[0]

    pre_period_cumulative_return = daily_data[daily_data['date'] < outlier_date]['daily_return'].sum()
    post_period_cumulative_return = daily_data[daily_data['date'] >= outlier_date]['daily_return'].sum()

    pre_threshold_return = mean_daily_return * pre_threshold
    post_threshold_return = mean_daily_return * post_threshold

    if pre_period_cumulative_return > pre_threshold_return:
            pre_recovery_classification = 'fast recovery'
    else:
        pre_recovery_classification = 'slow recovery'

    if post_period_cumulative_return > post_threshold_return:
        post_recovery_classification = 'fast recovery'
    else:
        post_recovery_classification = 'slow recovery'

    return pre_recovery_classification, post_recovery_classification

def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    df['abs_daily_return'] = df['daily_return'].abs()
    df['pct_change'] = df['c'].pct_change() * 100

    df['direction'] = df['pct_change'].apply(lambda x: 'Up' if x > 0 else 'Down')
    df['pct_change'] = df['pct_change'].abs()

    return df

In [15]:
def fetch_hourly_data_chunk(symbol, start_date, end_date, api_key):
    formatted_start_date = start_date.strftime('%Y-%m-%d')
    formatted_end_date = end_date.strftime('%Y-%m-%d')

    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/hour/{formatted_start_date}/{formatted_end_date}?apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return None
    
    response_data = response.json()
    
    if 'results' not in response_data:
        print(f"No 'results' in response: {response_data}")
        return None

    df = pd.DataFrame(response_data['results'])
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    
    return df

def fetch_and_process_hourly_data(symbol, start_date, end_date, api_key, outlier_date):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

    # Adjust start_date and end_date based on the largest day difference
    adjusted_start_date = outlier_date - pd.Timedelta(days=max_days_difference)
    adjusted_end_date = outlier_date + pd.Timedelta(days=max_days_difference)

    # Split the date range into smaller chunks
    chunk_size = 3  # Fetch data in 3-day chunks
    date_ranges = [(adjusted_start_date + timedelta(days=i*chunk_size),
                    min(adjusted_end_date, adjusted_start_date + timedelta(days=(i+1)*chunk_size - 1)))
                   for i in range((adjusted_end_date - adjusted_start_date).days // chunk_size + 1)]

    all_data = []

    for start, end in date_ranges:
        chunk_data = fetch_hourly_data_chunk(symbol, start, end, api_key)
        if chunk_data is not None:
            all_data.append(chunk_data)

    if not all_data:
        print("No data fetched")
        return None

    df = pd.concat(all_data)
    hourly_data = calculate_daily_returns(df)

    pre_recovery, post_recovery = classify_recovery_range(hourly_data, outlier_date, mean_daily_return)
    hourly_data.loc[hourly_data['date'] < outlier_date, 'recovery'] = pre_recovery
    hourly_data.loc[hourly_data['date'] >= outlier_date, 'recovery'] = post_recovery
    hourly_data.set_index('date', inplace=True)

    full_index = pd.date_range(start=adjusted_start_date, end=adjusted_end_date + timedelta(days=1), freq='H')
    hourly_data = hourly_data.reindex(full_index)

    hourly_data.reset_index(inplace=True)
    hourly_data.rename(columns={'index': 'date'}, inplace=True)

    return hourly_data

In [16]:
# Initialize empty DataFrame
all_data_daily = pd.DataFrame()

# Initialize an outlier identifier starting from 1 or any specific number
outlier_id = 1

for index, row in date_ranges.iterrows():
    # Convert outlier_date to Timestamp for consistent comparison
    outlier_date_ts = pd.Timestamp(row['outlier_date'])

    # Calculate the start and end dates for fetching hourly data based on max_days_difference
    adjusted_start_date_ts = outlier_date_ts - pd.Timedelta(days=max_days_difference)
    adjusted_end_date_ts = outlier_date_ts + pd.Timedelta(days=max_days_difference)

    # Get hourly data for the range including adjusted start and end dates
    daily_data = fetch_and_process_hourly_data(pair, adjusted_start_date_ts, adjusted_end_date_ts, api_key, outlier_date_ts)

    # Check if hourly_data is not None before processing
    if daily_data is not None:
        # Assign the current outlier_id to the data
        daily_data['outlier_id'] = outlier_id

        # Filter out weekends
        daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5,6])]

        # prior_data from adjusted_start_date to outlier_date inclusive
        prior_data = daily_data[(daily_data['date'] >= adjusted_start_date_ts) & (daily_data['date'] < outlier_date_ts)]
        prior_data["day type"] = "prior day"

        # outlier_data is for the hourly data on the day of the outlier
        outlier_data = daily_data[(daily_data['date'].dt.date == outlier_date_ts.date())]
        outlier_data["day type"] = "outlier day"

        # post_data from the day after outlier_date to adjusted_end_date
        post_outlier_ts = outlier_date_ts + pd.Timedelta(days=1)  # Starting the day after the outlier_date
        post_data = daily_data[(daily_data['date'] >= post_outlier_ts) & (daily_data['date'] <= adjusted_end_date_ts)]
        post_data["day type"] = "post day"

        # Concatenate the data from this iteration to the cumulative DataFrame
        all_data_daily = pd.concat([all_data_daily, prior_data, outlier_data, post_data])

        # Increment the outlier_id for the next iteration
        outlier_id += 1

    else:
        print(f"Data not available for symbol {pair} from {row['start_date'].date()} to {row['end_date'].date()}")

# Add the day column to the final DataFrame
all_data_daily['day'] = all_data_daily['date'].dt.day_name()
# Optionally, you can reset the index of the final DataFrame if it becomes non-unique after concatenations
all_data_daily.reset_index(drop=True, inplace=True)

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/216526559.py:58: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=adjusted_start_date, end=adjusted_end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/1105289910.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/1105289910.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [17]:
# Reset the index to have simple numbers and separate "date" and "time" columns
all_data_daily.reset_index(inplace=True)
all_data_daily.rename(columns={'index': 'New'}, inplace=True)

# Convert date to string format if necessary
all_data_daily['date'] = all_data_daily['date'].astype(str)

# Extract date and time from the date string
all_data_daily['Date'] = all_data_daily['date'].str[:10]
all_data_daily['Time'] = all_data_daily['date'].str[11:19]

# Drop the 'date' column if it's no longer needed
all_data_daily.drop(columns=['date', 'New', 'v'], inplace=True)

In [18]:
len(all_data_daily)

38781

In [19]:
all_data_daily.head()

,vw,o,c,h,l,n,prev_close,daily_return,abs_daily_return,pct_change,direction,recovery,outlier_id,day type,day,Date,Time
0,1.1342,1.13310,1.13420,1.13550,1.13256,5562.0,1.13310,0.000971,0.000971,0.097079,Up,fast recovery,1,prior day,Monday,2010-05-17,00:00:00
1,1.1340,1.13420,1.13438,1.13527,1.13275,5581.0,1.13420,0.000159,0.000159,0.015870,Up,fast recovery,1,prior day,Monday,2010-05-17,01:00:00
2,1.1403,1.13440,1.14112,1.14450,1.13438,11640.0,1.13438,0.005942,0.005942,0.594157,Up,fast recovery,1,prior day,Monday,2010-05-17,02:00:00
3,1.1411,1.14108,1.14149,1.14284,1.13929,5978.0,1.14112,0.000324,0.000324,0.032424,Up,fast recovery,1,prior day,Monday,2010-05-17,03:00:00
4,1.1405,1.14149,1.13926,1.14168,1.13918,4710.0,1.14149,-0.001954,0.001954,0.195359,Down,fast recovery,1,prior day,Monday,2010-05-17,04:00:00


In [20]:
missing_values = all_data_daily.isnull()
missing_summary = all_data_daily.isnull().sum()

# Display summary of missing values
print("\nSummary of Missing Values:")
print(missing_summary)


Summary of Missing Values:
vw                  1203
o                   1203
c                   1203
h                   1203
l                   1203
n                   1203
prev_close          1301
daily_return        1301
abs_daily_return    1301
pct_change          1301
direction           1203
recovery            1203
outlier_id             0
day type               0
day                    0
Date                   0
Time                   0
dtype: int64


In [21]:
all_data_daily.fillna(method='ffill', inplace=True)
all_data_daily.fillna(method='bfill', inplace=True)

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/1621982748.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data_daily.fillna(method='ffill', inplace=True)
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_13158/1621982748.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data_daily.fillna(method='bfill', inplace=True)


In [22]:
all_data_daily.head()

,vw,o,c,h,l,n,prev_close,daily_return,abs_daily_return,pct_change,direction,recovery,outlier_id,day type,day,Date,Time
0,1.1342,1.13310,1.13420,1.13550,1.13256,5562.0,1.13310,0.000971,0.000971,0.097079,Up,fast recovery,1,prior day,Monday,2010-05-17,00:00:00
1,1.1340,1.13420,1.13438,1.13527,1.13275,5581.0,1.13420,0.000159,0.000159,0.015870,Up,fast recovery,1,prior day,Monday,2010-05-17,01:00:00
2,1.1403,1.13440,1.14112,1.14450,1.13438,11640.0,1.13438,0.005942,0.005942,0.594157,Up,fast recovery,1,prior day,Monday,2010-05-17,02:00:00
3,1.1411,1.14108,1.14149,1.14284,1.13929,5978.0,1.14112,0.000324,0.000324,0.032424,Up,fast recovery,1,prior day,Monday,2010-05-17,03:00:00
4,1.1405,1.14149,1.13926,1.14168,1.13918,4710.0,1.14149,-0.001954,0.001954,0.195359,Down,fast recovery,1,prior day,Monday,2010-05-17,04:00:00


In [23]:
# Export the all_data_daily DataFrame to a CSV file
all_data_daily.to_csv('new_data.csv', index=False)

# Displaying the first few rows
all_data_daily.head()

,vw,o,c,h,l,n,prev_close,daily_return,abs_daily_return,pct_change,direction,recovery,outlier_id,day type,day,Date,Time
0,1.1342,1.13310,1.13420,1.13550,1.13256,5562.0,1.13310,0.000971,0.000971,0.097079,Up,fast recovery,1,prior day,Monday,2010-05-17,00:00:00
1,1.1340,1.13420,1.13438,1.13527,1.13275,5581.0,1.13420,0.000159,0.000159,0.015870,Up,fast recovery,1,prior day,Monday,2010-05-17,01:00:00
2,1.1403,1.13440,1.14112,1.14450,1.13438,11640.0,1.13438,0.005942,0.005942,0.594157,Up,fast recovery,1,prior day,Monday,2010-05-17,02:00:00
3,1.1411,1.14108,1.14149,1.14284,1.13929,5978.0,1.14112,0.000324,0.000324,0.032424,Up,fast recovery,1,prior day,Monday,2010-05-17,03:00:00
4,1.1405,1.14149,1.13926,1.14168,1.13918,4710.0,1.14149,-0.001954,0.001954,0.195359,Down,fast recovery,1,prior day,Monday,2010-05-17,04:00:00
